In [43]:
# pip install -r dependencies.txt

In [44]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import time

In [45]:
# Carregar dados CSV
diplomatas = pd.read_csv('resources/aed_alocacao_recursos - diplomatas.csv')
cidades = pd.read_csv('resources/aed_alocacao_recursos - cidades.csv')

# print(diplomatas.info())
# print(cidades.info())

In [46]:
def get_posto_trabalho(id, cidades):
  return cidades.loc[cidades['posto'] == id].to_dict(orient='records')[0]

get_posto_trabalho('P1', cidades)

{'cidade': 'C1', 'posto': 'P1', 'classificacao': 'A'}

In [93]:
def calcular_custo_aresta(diplomata, posto, cidades):
    # print(f"Calculando custo da aresta {diplomata.id} -> {posto.posto}")
    # Se tiver 12 anos fora, deve retornar
    if diplomata.tempo_fora >= 144:
        if posto.classificacao == '*':
            return 0.0
        else:
            return 999999999.99
    # Se tiver 2 anos na lotação atual, deve trocar de lotacao
    if diplomata.tempo_na_lotacao >= 24 and diplomata.lotacao == posto.posto:
        return 999999999.99

    # verificar se a origem á compatível com o destino. Ex. a só pode ir para B, C ou D
    if diplomata.lotacao != 'P0':
        posto_atual = get_posto_trabalho(diplomata.lotacao, cidades)
        # print('Posto atual {}'.format(posto_atual))
        if diplomata.lotacao != posto.posto and posto.classificacao == 'A':
            if posto_atual.get('classificacao')  == 'A':
                return 999999999.99
    elif diplomata.pedagio < 24 and posto.classificacao == 'A':
        # Se estiver a menos de 2 anos no Brasil, não pode ir para um posto 'A'
        return 999999999.99

    if posto.classificacao == 'A':
        return 0.0

    if posto.classificacao == 'B':
        return 10.0

    if posto.classificacao == 'C':
        return 100.0

    if posto.classificacao == 'D':
        return 1000.0
    
    return 999999999.99

In [48]:
def alocar_diplomatas(matriz_custos):
  """
  Resolve o problema de alocação usando o algoritmo húngaro.

  :param matriz_custos: Matriz de custos (2D list ou numpy array)
  :return: Tupla (lista de alocações, custo total)
  """
  # Usando a função `linear_sum_assignment` da biblioteca scipy
  matriz_custos = np.array(matriz_custos)
  linhas, colunas = linear_sum_assignment(matriz_custos)
  custo_total = matriz_custos[linhas, colunas].sum()

  return linhas, colunas, custo_total

In [49]:
def calcular_custos_arestas(diplomatas, cidades):
    diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
    pesos_arestas = []
    for d, row in diplomatas_tratado.iterrows():
        pesos_diplomata = []
        for i, posto in cidades.iterrows():
            pesos_diplomata.append(calcular_custo_aresta(row,posto,cidades))
        pesos_arestas.append(pesos_diplomata)

    return pesos_arestas

In [ ]:
# Exemplo de uso
pesos_arestas = calcular_custos_arestas(diplomatas,cidades)
linhas, colunas, custo = alocar_diplomatas(pesos_arestas)
diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
print("Alocações (diplomata -> posto):")
for i, j in zip(linhas, colunas):
  print(f"Diplomata {diplomatas_tratado.iloc[i].id} -> Posto {cidades.iloc[j].posto}")

print("Custo total:", custo)

In [86]:
def gerar_base_diplomatas(qtd_diplomatas, qtd_postos):
  # Geração da lotação
  ids = []
  pedagios = []
  lotacoes = []
  tempos_servico = []
  tempos_lotacao = []
  tempos_fora = []

  for p in range(qtd_diplomatas):
    while True:
      lotacao = 'P' + str(np.random.randint(qtd_postos))
      if lotacao == 'P0' or lotacao not in lotacoes:
        break

    tempo_servico = np.random.randint(300)  # tempo de serviço em meses
    pedagio = 0                             # tempo no Brasil em anos
    tempo_na_lotacao = 0                    # tempo na lotação em meses
    tempo_fora = 0                          # tempo ininterrupro fora do Brasil em meses
    if lotacao == 'P0': # Ele está no Brasil
      pedagio = np.random.randint(30)
      tempo_na_lotacao = 0
      tempo_fora = 0
    else:
      tempo_na_lotacao = np.random.randint(25)
      tempo_fora = tempo_na_lotacao + np.random.randint(96,121)
      pedagio = 0

    ids.append('D' + str(p+1))
    pedagios.append(pedagio)
    lotacoes.append(lotacao)
    tempos_servico.append(tempo_servico)
    tempos_lotacao.append(tempo_na_lotacao)
    tempos_fora.append(tempo_fora)

  d = {
    'id': ids,
    'pedagio': pedagios,
    'lotacao': lotacoes,
    'tempo_servico': tempos_servico,
    'tempo_na_lotacao': tempos_lotacao,
    'tempo_fora': tempos_fora
  }
  return pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])

In [ ]:
# dpls = gerar_base_diplomatas(150,175)

# dpls.loc[(dpls['lotacao'] != 'P0') & (dpls['pedagio'] != 0)]

,id,pedagio,lotacao,tempo_servico,tempo_na_lotacao,tempo_fora


In [52]:
def gerar_base_postos(qtd_postos):
  postos = ['P0']
  classificacoes = ['*']
  tipos_classificacoes = ['A','B','C','D']

  for p in range(qtd_postos):
    posto = 'P' + str(p+1)
    postos.append(posto)
    classificacoes.append(tipos_classificacoes[np.random.randint(4)])

  d = {
    'posto': postos,
    'classificacao': classificacoes
  }

  return pd.DataFrame(data=d, index=[*range(qtd_postos+1)])

In [70]:
def rodar_teste(qtd_diplomatas, multiplicador_postos = 1.5):
  # Gerar base de testes
  qtd_postos = multiplicador_postos * qtd_diplomatas
  t_postos = gerar_base_postos(int(qtd_postos))
  t_diplomatas = gerar_base_diplomatas(qtd_diplomatas, qtd_postos)

  # Executar alocações
  inicio = time.time()
  t_pesos_arestas = calcular_custos_arestas(t_diplomatas,t_postos)
  linhas, colunas, custo = alocar_diplomatas(t_pesos_arestas)
  fim = time.time()
  #preparar dados de retorno
  # pedagio,lotacao,tempo_servico,tempo_na_lotacao,tempo_fora
  t_diplomatas_tratado = t_diplomatas.sort_values("tempo_servico", ascending = False)
  t_diplomatas_tratado = t_diplomatas_tratado.merge(t_postos, how='inner', left_on='lotacao',right_on='posto')
  t_diplomatas_alocados = []
  t_postos_alocados = []
  t_classificacoes_atuais = []
  t_classificacoes_alocadas = []
  t_tempos_servicos = []
  t_pedagios = []
  t_lotacoes_anteriores = []
  t_tempos_nas_lotacoes = []
  t_tempos_fora = []

  for i, j in zip(linhas, colunas):
    # print(f"Diplomata {t_diplomatas_tratado.iloc[i].id} -> Posto {t_postos.iloc[j].posto}")
    t_diplomatas_alocados.append(t_diplomatas_tratado.iloc[i].id)
    t_tempos_servicos.append(t_diplomatas_tratado.iloc[i].tempo_servico)
    t_tempos_nas_lotacoes.append(t_diplomatas_tratado.iloc[i].tempo_na_lotacao)
    t_tempos_fora.append(t_diplomatas_tratado.iloc[i].tempo_fora)
    t_pedagios.append(t_diplomatas_tratado.iloc[i].pedagio)
    t_lotacoes_anteriores.append(t_diplomatas_tratado.iloc[i].lotacao)
    t_postos_alocados.append(t_postos.iloc[j].posto)
    t_classificacoes_atuais.append(t_diplomatas_tratado.iloc[i].classificacao)
    t_classificacoes_alocadas.append(t_postos.iloc[j].classificacao)

  d= {
    'diplomata': t_diplomatas_alocados,
    'tempo_servico': t_tempos_servicos,
    'tempo_na_lotacao': t_tempos_nas_lotacoes,
    'tempo_fora': t_tempos_fora,
    'tempo_sede': t_pedagios,
    'lotacao_anterior': t_lotacoes_anteriores,
    'classificacao_anterior': t_classificacoes_atuais,
    'lotacao_alocada': t_postos_alocados,
    'classificacao_alocada': t_classificacoes_alocadas
  }
  df = pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])
  tempo = fim - inicio
  # print("Custo total:", custo)
  # print(f"Tempo de execução igual a {fim - inicio}")
  return df, custo, tempo

In [88]:
df, custo, tempo =  rodar_teste(150)

# dplm = gerar_base_diplomatas(100,150)
# pst = gerar_base_postos(150)

# all = dplm.merge(pst, how='inner', left_on='lotacao',right_on='posto')



In [92]:
# média de tempo de serviço
# float(dplm['tempo_servico'].mean())/12

# Mediana do tempo de serviço
# float(dplm['tempo_servico'].median())/12

# dplm[['tempo_servico', 'tempo_na_lotacao', 'tempo_fora', 'pedagio']].describe()
# dplm[["posto","tempo_servico"]].groupby()

#####################
#### Calular a média de tempo de serviço por classe de posto antes e depois
####
####################
# print(df[['classificacao_anterior', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_anterior').mean())
# print(df[['classificacao_alocada', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_alocada').mean())

df.loc[df['classificacao_alocada']=='*']

,diplomata,tempo_servico,tempo_na_lotacao,tempo_fora,tempo_sede,lotacao_anterior,classificacao_anterior,lotacao_alocada,classificacao_alocada
